# SBTi Temperature Alignment tool examples
This notebook provides some examples on how to use this module. It uses autoreloading to ease development. If you'd like to employ this the SBTi package should be installed in development mode. This can be done using the following command:

```bash
python setup.py develop
```

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2
import SBTi
from SBTi.data.excel import ExcelProvider
from SBTi.portfolio_aggregation import PortfolioAggregationMethod
from SBTi.portfolio_coverage_tvp import PortfolioCoverageTVP
from SBTi.temperature_score import TemperatureScore, Scenario
from SBTi.target_valuation_protocol import TargetValuationProtocol
%aimport -pandas
import pandas as pd

### Create a data provider
Data providers let you connect to the data source of your choice.

In [3]:
provider = ExcelProvider(path="data/data_provider_example.xlsx")

### Load your portfolio
In our case the portfolio is stored as a CSV file. The portfolio should at least have an "id" (the identifier of the company) and a "proportion" (the weight of the company in your portfolio e.g. the value of the shares you hold) column.

In [4]:
portfolio = pd.read_csv("example_portfolio.csv", encoding="iso-8859-1")

In [5]:
portfolio.head(5)

,company_name,company_id,ISIN,weights,investment_value,engagement_type
0,"Advanced Micro Devices, Inc",US0079031078,US0079031078,6.36%,35000000,set_targets
1,Adobe Systems Inc.,US00724F1012,US00724F1012,1.82%,10000000,set_targets
2,Capgemini Group,FR0000125338,FR0000125338,1.82%,10000000,set_targets
3,"Cisco Systems, Inc.",US17275R1023,US17275R1023,1.82%,10000000,set_targets
4,Coca-Cola HBC AG,CH0198251305,CH0198251305,1.82%,10000000,set_targets


To load the data from the data provider, we have to pass a list of dictionaries, containing a "company_name" and "company_id".

In [6]:
companies = portfolio[["company_id", "company_name"]].to_dict(orient="records")

### Load the data
We'll call the data provider and ask it to provide us with the targets and the company data.

In [7]:
company_data = SBTi.data.get_company_data([provider], companies)
targets = SBTi.data.get_targets([provider], companies)
portfolio_data = pd.merge(left=company_data, right=targets, how='outer', on=['company_name', 'company_id'])

### Validate the targets
We've now got our portfolio, amended with target data. Now we need to make sure that all the targets are valid (e.g. they haven't been reached yet and their end date isn't in the past).

In [9]:
target_valuation_protocol = TargetValuationProtocol(portfolio_data, company_data)
portfolio_data = target_valuation_protocol.target_valuation_protocol()

### Add the portfolio-related information
This information has been dropped in the previous step. We have to re-add it here.

In [10]:
portfolio_data = pd.merge(left=portfolio_data, right=portfolio, how='outer', on=['company_name', 'company_id'])

### Calculate the temperature scores
In the amended portfolio you'll find your original portfolio, amended with the emissions and the temperature score.

In [12]:
temperature_score = TemperatureScore()
amended_portfolio = temperature_score.calculate(portfolio_data, ["investment_value"])

In [13]:
amended_portfolio.head(10)

,company_name,company_id,ISIC,Country,Region,Industry_lvl1,Industry_lvl2,Industry_lvl3,Industry_lvl4,sector,...,time_frame,ISIN,weights,investment_value,engagement_type,SR15,annual_reduction_rate,regression_param,regression_intercept,temperature_score
0,"Advanced Micro Devices, Inc",US0079031078,NaN,United States,North America,NaN,NaN,NaN,NaN,Industrials,...,short,US0079031078,6.36%,35000000,set_targets,Emissions|Kyoto Gases,0.060000,-0.212692,2.574303,1.298150
1,"Advanced Micro Devices, Inc",US0079031078,NaN,United States,North America,NaN,NaN,NaN,NaN,Industrials,...,short,US0079031078,6.36%,35000000,set_targets,Emissions|Kyoto Gases,0.131643,-0.212692,2.574303,-0.225637
2,"Advanced Micro Devices, Inc",US0079031078,NaN,United States,North America,NaN,NaN,NaN,NaN,Industrials,...,mid,US0079031078,6.36%,35000000,set_targets,Emissions|Kyoto Gases,0.027785,-0.312329,2.697261,1.829468
3,"Advanced Micro Devices, Inc",US0079031078,NaN,United States,North America,NaN,NaN,NaN,NaN,None,...,mid,US0079031078,6.36%,35000000,set_targets,Emissions|Kyoto Gases,NaN,-0.312329,2.697261,3.200000
4,"Advanced Micro Devices, Inc",US0079031078,NaN,United States,North America,NaN,NaN,NaN,NaN,None,...,long,US0079031078,6.36%,35000000,set_targets,Emissions|Kyoto Gases,NaN,-0.483331,2.841338,3.200000
5,"Advanced Micro Devices, Inc",US0079031078,NaN,United States,North America,NaN,NaN,NaN,NaN,None,...,long,US0079031078,6.36%,35000000,set_targets,Emissions|Kyoto Gases,NaN,-0.483331,2.841338,3.200000
6,Adobe Systems Inc.,US00724F1012,NaN,United States,North America,NaN,NaN,NaN,NaN,None,...,short,US00724F1012,1.82%,10000000,set_targets,Emissions|Kyoto Gases,NaN,-0.212692,2.574303,3.200000
7,Adobe Systems Inc.,US00724F1012,NaN,United States,North America,NaN,NaN,NaN,NaN,None,...,short,US00724F1012,1.82%,10000000,set_targets,Emissions|Kyoto Gases,NaN,-0.212692,2.574303,3.200000
8,Adobe Systems Inc.,US00724F1012,NaN,United States,North America,NaN,NaN,NaN,NaN,Utilities,...,mid,US00724F1012,1.82%,10000000,set_targets,Emissions|Kyoto Gases,0.032559,-0.312329,2.697261,1.680349
9,Adobe Systems Inc.,US00724F1012,NaN,United States,North America,NaN,NaN,NaN,NaN,Utilities,...,mid,US00724F1012,1.82%,10000000,set_targets,Emissions|Kyoto Gases,0.018000,-0.312329,2.697261,2.135069


### Calculate the aggregated temperature score
Calculate an aggregated temperature score. This can be done use different aggregation methods. Here we'll use the "Weighted Average Temperature Score". For more details, have a look at notebook 3 ("3_portfolio_aggregations.ipynb"). The termperature scores are calculated per time-frame/scope combination.

In [31]:
def print_aggregations(aggregations):
    for time_frame, time_frame_values in aggregations.items():
        for scope, scope_values in time_frame_values.items():
            print("{} - {}: {} degrees celcius".format(time_frame, scope, scope_values["all"]["score"]))

#### WATS
Weighted Average Temperature Score (WATS): Temperature scores are allocated based on portfolio weights.
This requires the "investment_value" field to be defined in your data set.

In [34]:
print_aggregations(temperature_score.aggregate_scores(amended_portfolio, PortfolioAggregationMethod.WATS, []))

short - s1s2: 1.4629 degrees celcius
short - s3: 2.3409 degrees celcius
short - s1s2s3: 2.0977 degrees celcius
mid - s1s2: 2.5923 degrees celcius
mid - s3: 2.9923 degrees celcius
mid - s1s2s3: 2.8998 degrees celcius
long - s1s2: 3.0814 degrees celcius
long - s3: 3.191 degrees celcius
long - s1s2s3: 3.155 degrees celcius


#### TETS
Total emissions weighted temperature score (TETS): Temperature scores are allocated based on historical emission weights using total company emissions. 

In [35]:
print_aggregations(temperature_score.aggregate_scores(amended_portfolio, PortfolioAggregationMethod.TETS, []))

short - s1s2: 1.6343 degrees celcius
short - s3: 2.3879 degrees celcius
short - s1s2s3: 2.0774 degrees celcius
mid - s1s2: 2.681 degrees celcius
mid - s3: 2.3564 degrees celcius
mid - s1s2s3: 2.5294 degrees celcius
long - s1s2: 2.8646 degrees celcius
long - s3: 3.1986 degrees celcius
long - s1s2s3: 3.0562 degrees celcius


#### WATS
Weighted Average Temperature Score (WATS): Temperature scores are allocated based on portfolio weights.
This requires the "investment_value" field to be defined in your data set.

#### WATS
Weighted Average Temperature Score (WATS): Temperature scores are allocated based on portfolio weights.
This requires the "investment_value" field to be defined in your data set.

#### WATS
Weighted Average Temperature Score (WATS): Temperature scores are allocated based on portfolio weights.
This requires the "investment_value" field to be defined in your data set.

#### WATS
Weighted Average Temperature Score (WATS): Temperature scores are allocated based on portfolio weights.
This requires the "investment_value" field to be defined in your data set.

#### WATS
Weighted Average Temperature Score (WATS): Temperature scores are allocated based on portfolio weights.
This requires the "investment_value" field to be defined in your data set.

### Portfolio coverage

Gives insights in the coverage of portfolio entities that has set GHG emissions reduction targets, STBi-validated or otherwise. Companies with SBTi status Approved have a score of 100%, all others have a score of 0%. 

This can be done use different aggregation methods. Here we'll use the "Weighted Average Temperature Score". For more details, have a look at notebook 3 ("3_portfolio_aggregations.ipynb").

In [14]:
portfolio_coverage_tvp = PortfolioCoverageTVP()
coverage = portfolio_coverage_tvp.get_portfolio_coverage(amended_portfolio.copy(), PortfolioAggregationMethod.WATS)
round(coverage * 100) / 100

35.45